## EEG Cross-Dataset Preprocessing Pipeline

This notebook loads and preprocesses two EEG datasets (`Rest_state` and `PD_IntervalTiming`) that share the same channel layout. It applies a standardized preprocessing pipeline:

- **Loading & organization** for both datasets
- **Resampling, band-pass filtering, CAR re-referencing**
- **ICA-based artifact removal** (eye blinks, muscle)
- **Bad channel detection and interpolation**
- **Per-subject z-score normalization**
- **Riemannian re-centering** of covariance features for domain alignment

> Note: Make sure you have `mne`, `mne-bids`, `pyriemann`, and `numpy`, `pandas`, `scikit-learn` installed in your environment.


In [25]:
# 1. Imports and basic configuration

import os
from pathlib import Path

import numpy as np
import pandas as pd

import mne
from mne.preprocessing import ICA, create_eog_epochs

# Optional: pyRiemann for Riemannian geometry
from pyriemann.utils.mean import mean_riemann
from pyriemann.tangentspace import TangentSpace

# Display MNE log more compactly
mne.set_log_level("INFO")

# Set base paths (relative to your project root)
# Adjust these if your folder names differ
BASE_DIR = Path(".")
REST_DIR = BASE_DIR  / "ds004584-download"
PD_DIR = BASE_DIR / "JamesCavanagh"/"PD_Dataset_timing"

# Preprocessing parameters (shared across datasets)
TARGET_SFREQ = 250           # Hz (you can change to 500 if you prefer)
L_FREQ = 0.5                 # Hz (high-pass)
H_FREQ = 50.0                # Hz (low-pass)

# Channels that are EOG (if present) to help ICA find eye components
EOG_CHANNELS = ["VEOG", "HEOG", "EOG", "Fp1", "Fp2"]  # adapt to your montage


In [26]:
from pathlib import Path

print("PD_DIR =", PD_DIR)
print("Found .vhdr files:")
for f in Path(PD_DIR).glob("*.vhdr"):
    print("  ", f.name)

PD_DIR = JamesCavanagh\PD_Dataset_timing
Found .vhdr files:
   Control1025.vhdr
   Control1035.vhdr
   Control1055.vhdr
   Control1065.vhdr
   Control1075.vhdr
   Control1085.vhdr
   Control1095.vhdr
   Control1105.vhdr
   Control1115.vhdr
   Control1125.vhdr
   Control1135.vhdr
   Control1145.vhdr
   Control1155.vhdr
   Control1175.vhdr
   Control1185.vhdr
   Control1195.vhdr
   Control1205.vhdr
   Control1215.vhdr
   Control1225.vhdr
   Control1235.vhdr
   Control1245.vhdr
   Control1255.vhdr
   Control1265.vhdr
   Control1275.vhdr
   Control1285.vhdr
   Control1295.vhdr
   Control1305.vhdr
   Control1315.vhdr
   Control1325.vhdr
   Control1335.vhdr
   Control1345.vhdr
   Control1365.vhdr
   Control1375.vhdr
   Control1385.vhdr
   Control1395.vhdr
   Control1405.vhdr
   Control1415.vhdr
   PD1005.vhdr
   PD1015.vhdr
   PD1025.vhdr
   PD1035.vhdr
   PD1045.vhdr
   PD1055.vhdr
   PD1065.vhdr
   PD1075.vhdr
   PD1085.vhdr
   PD1095.vhdr
   PD1105.vhdr
   PD1115.vhdr
   PD1125.vhdr
   PD

In [27]:
PD_DIR = BASE_DIR / "JamesCavanagh" / "PD_Dataset_timing"

In [28]:
# 2. Helper functions: loading for both datasets


def load_rest_state_subject(sub_id: str) -> mne.io.BaseRaw:
    """Load one subject from the Rest_state BIDS-like folder.

    Expects files like:
    Rest_state/ds004584-download/sub-001/eeg/sub-001_task-Rest_eeg.set
    """
    sub_folder = REST_DIR / f"sub-{sub_id}"
    eeg_dir = sub_folder / "eeg"

    # If there is no explicit `eeg` folder, fall back to subject root
    if not eeg_dir.exists():
        eeg_dir = sub_folder

    set_files = list(eeg_dir.glob(f"sub-{sub_id}_task-Rest_eeg.set"))
    if len(set_files) == 0:
        raise FileNotFoundError(f"No .set file found for subject {sub_id} in {eeg_dir}")

    raw = mne.io.read_raw_eeglab(set_files[0], preload=True)
    return raw


def load_pd_interval_subject(stem: str) -> mne.io.BaseRaw:
    """Load one subject from the PD_IntervalTiming BrainVision files.

    Example stem: 'control2931' or 'PD1032'
    Expects files like:
    PD_IntervalTiming/control2931.vhdr
    """
    vhdr_path = PD_DIR / f"{stem}.vhdr"
    if not vhdr_path.exists():
        raise FileNotFoundError(f"No .vhdr file found for stem {stem} in {PD_DIR}")

    raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
    return raw


# Example lists of subjects; adjust these to match your data
rest_subjects = ["001", "002"]  # extend based on Rest_state/participants.tsv
pd_subjects = ["control2931", "PD1032"]  # stems of .vhdr files


In [12]:
# 3. Standardized preprocessing functions


def apply_standard_preprocessing(raw: mne.io.BaseRaw,
                                 target_sfreq: float = TARGET_SFREQ,
                                 l_freq: float = L_FREQ,
                                 h_freq: float = H_FREQ) -> mne.io.BaseRaw:
    """Apply resampling, band-pass filtering, CAR re-referencing.

    This function is applied identically to both datasets.
    """
    raw = raw.copy()

    # Ensure EEG channel types are correctly set
    raw.pick_types(eeg=True, eog=True, stim=False, meg=False, misc=True)

    # Resample
    if raw.info["sfreq"] != target_sfreq:
        raw.resample(target_sfreq)

    # Band-pass filter
    raw.filter(l_freq=l_freq, h_freq=h_freq, fir_design="firwin")

    # Set common average reference (CAR)
    raw.set_eeg_reference("average", projection=False)

    return raw


def detect_bad_channels(raw: mne.io.BaseRaw, z_thresh: float = 3.0):
    """Simple automatic bad channel detection based on variance z-score.

    You can replace this with MNE's built-in auto-detection (e.g., autoreject)
    for more robust behavior.
    """
    # For Raw objects, get_data() returns only the data array when return_times=False (default)
    data = raw.get_data()
    chan_var = data.var(axis=1)
    z = (chan_var - np.mean(chan_var)) / np.std(chan_var)
    bad_idx = np.where(np.abs(z) > z_thresh)[0]
    bads = [raw.ch_names[i] for i in bad_idx]
    return bads


def run_ica_and_remove_artifacts(raw: mne.io.BaseRaw,
                                 n_components: int | None = None,
                                 random_state: int = 97,
                                 max_iter: str | int = "auto") -> tuple[mne.io.BaseRaw, ICA]:
    """Run ICA and remove components related to eye blinks / EOG.

    In a multi-dataset setting, you can additionally use ICLabel (MNE-ICLabel)
    to identify artifact components in a standardized way across subjects.
    """
    raw = raw.copy()

    # Prepare EOG channels if any are present
    existing_eog = [ch for ch in EOG_CHANNELS if ch in raw.ch_names]
    if existing_eog:
        for ch in existing_eog:
            mne.utils.logger.info(f"Using {ch} as EOG channel for ICA.")
    else:
        mne.utils.logger.info("No explicit EOG channels found; ICA will still run but EOG detection may be weaker.")

    ica = ICA(n_components=n_components, random_state=random_state, max_iter=max_iter, method="fastica")
    ica.fit(raw)

    eog_indices = []
    eog_scores = []
    for ch_name in existing_eog:
        inds, scores = ica.find_bads_eog(raw, ch_name=ch_name)
        eog_indices.extend(inds)
        eog_scores.extend(scores)

    # Remove duplicates
    eog_indices = list(sorted(set(eog_indices)))

    if len(eog_indices) > 0:
        ica.exclude = eog_indices
        mne.utils.logger.info(f"Excluding ICA components (EOG-related): {ica.exclude}")
        raw_clean = ica.apply(raw.copy())
    else:
        raw_clean = raw

    return raw_clean, ica


def interpolate_bad_channels(raw: mne.io.BaseRaw, bads: list[str]) -> mne.io.BaseRaw:
    """Mark and interpolate bad channels using MNE's spherical spline interpolation."""
    raw = raw.copy()

    # Set montage if not already present (required for interpolation)
    if raw.get_montage() is None:
        try:
            raw.set_montage("standard_1020")
        except Exception as e:
            mne.utils.logger.warning(f"Could not set standard_1020 montage automatically: {e}")

    raw.info["bads"] = list(set(raw.info.get("bads", []) + bads))
    raw.interpolate_bads(reset_bads=True)
    return raw


In [21]:
# 4. Subject-level z-scoring and Riemannian re-centering


def zscore_subject_raw(raw: mne.io.BaseRaw) -> mne.io.BaseRaw:
    """Z-score each EEG channel over time for one subject.

    This centers each channel to mean 0 and std 1, reducing hardware/scale effects.
    """
    raw = raw.copy()
    data, times = raw.get_data(return_times=True)

    mean = data.mean(axis=1, keepdims=True)
    std = data.std(axis=1, keepdims=True)
    std[std == 0] = 1.0

    data_z = (data - mean) / std
    raw_z = mne.io.RawArray(data_z, raw.info, first_samp=raw.first_samp)
    return raw_z


def compute_covariances(raw: mne.io.BaseRaw, epoch_length: float = 2.0) -> np.ndarray:
    """Epoch the continuous data and compute spatial covariance matrices.

    Returns an array of shape (n_epochs, n_channels, n_channels).
    """
    # Make fixed-length epochs from continuous data
    epochs = mne.make_fixed_length_epochs(raw, duration=epoch_length, preload=True, overlap=0.0)
    data = epochs.get_data()  # (n_epochs, n_channels, n_times)

    covs = np.array([np.cov(trial) for trial in data])
    return covs


def riemannian_recenter(covs: np.ndarray, reg: float = 1e-6) -> tuple[np.ndarray, np.ndarray]:
    """Recenter covariance matrices on the Riemannian manifold with regularization.

    - Adds `reg * trace(C) / n_channels` to the diagonal of each covariance matrix
      to ensure positive-definiteness.
    - Computes the geometric mean (Riemannian mean) of the regularized covariances.
    - Maps covariances to tangent space around this mean (whitening / centering).
    """
    covs = np.asarray(covs)
    n_epochs, n_channels, _ = covs.shape

    # Regularize each covariance to make it positive definite
    covs_reg = covs.copy()
    for i in range(n_epochs):
        tr = np.trace(covs_reg[i])
        covs_reg[i] += reg * (tr / n_channels) * np.eye(n_channels)

    C_ref = mean_riemann(covs_reg)
    # Older versions of pyRiemann do not support the `reference` argument in TangentSpace.
    # We therefore let TangentSpace estimate its own reference (which is also the Riemannian mean)
    # and still return C_ref separately for inspection.
    ts = TangentSpace(metric="riemann")
    tangent_features = ts.fit_transform(covs_reg)
    return tangent_features, C_ref

In [29]:
# 5. Full pipeline execution for both datasets

all_features = []
all_labels = []  # e.g., 0 = Rest_state, 1 = PD_IntervalTiming
all_subject_ids = []
all_dataset_names = []

# --- Rest_state dataset ---
for sub_id in rest_subjects:
    print(f"Processing Rest_state subject {sub_id}...")
    raw = load_rest_state_subject(sub_id)

    # Standard preprocessing
    raw = apply_standard_preprocessing(raw)

    # Bad channel detection & interpolation
    bads = detect_bad_channels(raw)
    if bads:
        print(f"  Detected bad channels: {bads}")
        raw = interpolate_bad_channels(raw, bads)

    # ICA artifact removal
    raw, ica = run_ica_and_remove_artifacts(raw)

    # Subject-level z-scoring
    raw = zscore_subject_raw(raw)

    # Compute covariance-based features and Riemannian re-centering
    covs = compute_covariances(raw, epoch_length=2.0)
    feats, C_ref = riemannian_recenter(covs)

    all_features.append(feats)
    all_labels.append(np.zeros(len(feats), dtype=int))  # 0 for Rest_state
    all_subject_ids.extend([f"Rest_{sub_id}"] * len(feats))
    all_dataset_names.extend(["Rest_state"] * len(feats))

# --- PD_IntervalTiming dataset ---
for stem in pd_subjects:
    print(f"Processing PD_IntervalTiming subject {stem}...")
    raw = load_pd_interval_subject(stem)

    # Standard preprocessing
    raw = apply_standard_preprocessing(raw)

    # Bad channel detection & interpolation
    bads = detect_bad_channels(raw)
    if bads:
        print(f"  Detected bad channels: {bads}")
        raw = interpolate_bad_channels(raw, bads)

    # ICA artifact removal
    raw, ica = run_ica_and_remove_artifacts(raw)

    # Subject-level z-scoring
    raw = zscore_subject_raw(raw)

    # Compute covariance-based features and Riemannian re-centering
    covs = compute_covariances(raw, epoch_length=2.0)
    feats, C_ref = riemannian_recenter(covs)

    all_features.append(feats)
    all_labels.append(np.ones(len(feats), dtype=int))  # 1 for PD_IntervalTiming
    all_subject_ids.extend([f"PD_{stem}"] * len(feats))
    all_dataset_names.extend(["PD_IntervalTiming"] * len(feats))

# Concatenate all features/labels into final arrays
X = np.vstack(all_features)
y = np.concatenate(all_labels)

print("Final feature matrix shape:", X.shape)
print("Final label vector shape:", y.shape)


Processing Rest_state subject 001...
Reading c:\Users\Usha Sri\OneDrive\Documents\Parkinson_Project\ds004584-download\sub-001\eeg\sub-001_task-Rest_eeg.fdt


Reading 0 ... 140829  =      0.000 ...   281.658 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_19368\2623207488.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_files[0], preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (6.604 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
  Detected bad channels: ['AF8']
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 100.0 mm
Computing interpolation matrix from 62 sensor positions
Interpolating 1 sensors
Using Fp1 as EOG channel for ICA.
Using Fp2 as EOG channel for ICA.
Fitting

C:\Users\Usha Sri\AppData\Local\Temp\ipykernel_19368\2623207488.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_files[0], preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (6.604 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
  Detected bad channels: ['P7']
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 100.0 mm
Computing interpolation matrix from 62 sensor positions
Interpolating 1 sensors
Using Fp1 as EOG channel for ICA.
Using Fp2 as EOG channel for ICA.
Fitting 

FileNotFoundError: No .vhdr file found for stem control2931 in JamesCavanagh\PD_Dataset_timing

## 6. Notes and next steps

- **Channel consistency**: This notebook assumes both datasets share the same EEG channel names and order. If they differ, you should create a channel-mapping step before preprocessing.
- **ICLabel usage**: For fully automatic ICA component labeling, install `mne-icalabel` and replace the manual EOG-based rejection with ICLabel-based rejection to keep the pipeline objective and consistent across datasets.
- **Bad-channel strategy**: The provided bad-channel detection is variance-based and simple. For production-quality pipelines, consider `autoreject` or visual inspection combined with ICLabel.
- **Feature usage**: The matrix `X` (Riemannian tangent-space features) and labels `y` are ready to be fed into a classifier (e.g., `sklearn` models) for tasks such as PD vs control classification or cross-dataset generalization.

You can now extend this notebook with model training, cross-validation, and domain adaptation experiments as needed.
